# Part 3: Chains and Runnables in LangChain

## Chains: A Look Back

Imagine LangChain in its early days, when developers were just starting to figure out how to build applications around Large Language Models (LLMs). There was a clear need to string together different operations. You'd want to take some user input, pass it through a prompt, then send that to an LLM, and maybe even process the LLM's output further. This is precisely what **Chains** were designed to do.

### What Chains Were and Their Original Purpose

In essence, **Chains were sequences of calls, or "links," that processed data in a predefined order.** Each link in the chain would take an input, perform an operation, and then pass its output as the input to the next link. Think of it like an assembly line, where each station performs a specific task.

The original purpose of Chains was to simplify the development of multi-step LLM applications. Instead of manually managing the input and output of each component, you could define a chain, and LangChain would handle the flow for you.

### Simple Chain Examples

Let's look at a very basic example of what a chain might have looked like. Remember, we're not going to run this code as Chains are largely deprecated, but it helps to understand the concept.

```python
# This is conceptual code to illustrate old Chains
# Do NOT run this code as it uses deprecated patterns

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain # Deprecated import

# 1. Define a Prompt Template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# 2. Define the LLM (hypothetically)
llm = OpenAI(temperature=0.7)

# 3. Create an LLMChain
# llm_chain = LLMChain(prompt=prompt, llm=llm) # This is how it used to be

# To use it:
# response = llm_chain.run("colorful socks")
# print(response)
```

In this conceptual example, the `LLMChain` took a prompt and an LLM, effectively chaining them together. You'd give it the product, it would format the prompt, send it to the LLM, and return the LLM's response.

You could even create more complex chains, for instance, by adding a simple parsing step:

```python
# This is conceptual code to illustrate old Chains
# Do NOT run this code as it uses deprecated patterns

# from langchain.chains import SimpleSequentialChain # Deprecated import

# prompt1 = PromptTemplate(...) # First prompt
# llm1 = OpenAI(...) # First LLM
# chain1 = LLMChain(prompt=prompt1, llm=llm1)

# prompt2 = PromptTemplate(...) # Second prompt
# llm2 = OpenAI(...) # Second LLM
# chain2 = LLMChain(prompt=prompt2, llm=llm2)

# overall_chain = SimpleSequentialChain(chains=[chain1, chain2]) # Chaining them
# response = overall_chain.run("some input")
```

Here, `SimpleSequentialChain` allowed you to run multiple chains in sequence, where the output of one became the input of the next.

### Key Limitations that Led to Their Replacement

While Chains were a good start, they had some significant limitations that became apparent as LangChain grew and the complexity of LLM applications increased:

1.  **Lack of Flexibility:** Chains were often rigid. If you wanted to do something that wasn't a simple sequential flow, like branching logic, parallel execution, or handling multiple inputs/outputs, it became cumbersome or impossible.
2.  **Limited Error Handling and Debugging:** Debugging complex chains could be challenging. It was often hard to pinpoint where an error occurred or to inspect intermediate results.
3.  **No Streaming or Async Support:** As LLMs became more responsive, the need for streaming outputs (like seeing words appear one by one) and asynchronous execution (running multiple things at once without blocking) became crucial. Chains weren't built with this in mind.
4.  **Poor Compositionality:** While they introduced the *concept* of composition, the way Chains were designed often led to a hierarchical, nested structure that was hard to reason about and modify. It wasn't always clear how to compose different types of operations effectively.
5.  **Performance Issues:** Without native support for concurrent execution, performance could suffer in scenarios where multiple LLM calls or other operations could theoretically run in parallel.

### How Chains Introduced the Concept of Component Composition

Despite their limitations, Chains were instrumental in introducing a fundamental idea that still underpins LangChain: **component composition**. They showed us the power of breaking down complex tasks into smaller, manageable pieces (like prompts, LLMs, parsers) and then assembling those pieces into a larger workflow. This modularity is key to building scalable and maintainable LLM applications.

### This is Where Runnables Come In...

The LangChain team recognized these limitations and completely reimagined how components should interact. They wanted a system that was more flexible, more powerful, and inherently supported modern application development patterns like streaming and asynchronicity.

And that, my friends, is why **Runnables** were born\! They are the evolution of component composition in LangChain, designed to address all the shortcomings of Chains and provide a robust framework for building almost any LLM application imaginable.

-----

### Introduction: The Building Blocks of LangChain

The best way to think of a **Runnable** is as a single "unit of work." Every major component in LangChain—from prompt templates to language models to output parsers—is a Runnable. This is what makes LangChain so modular and powerful. This standardized interface is the secret sauce that allows for such seamless composition.

This design philosophy means you can take these individual units of work and chain them together in creative ways to build complex applications. The "glue" that holds them all together is the **Runnable interface**, which provides a standard set of methods (`invoke`, `batch`, `stream`) to execute these units of work. Because every component speaks the same "language," you can be confident that they will connect predictably, allowing you to build robust and scalable AI systems.

### Core Types of Runnables

Here are practical examples of the four fundamental Runnable types we discussed.

#### 1. `RunnableLambda`

This is how you turn any regular Python function into a LangChain component that you can use in a chain. It's perfect for simple, custom transformations, data validation, or any ad-hoc logic you need to inject into a sequence. It provides an escape hatch to use traditional code within the declarative LangChain Expression Language (LCEL) framework.

**Use Case:** You want to create a simple step in your chain that takes a list of words and joins them into a single sentence.

```python
from langchain_core.runnables import RunnableLambda

# A simple python function
def join_words(words: list) -> str:
  """Takes a list of words and joins them with a space."""
  return " ".join(words)

# Wrap the function in a RunnableLambda
join_words_runnable = RunnableLambda(join_words)

# Now you can use it like any other LangChain component
input_words = ["LangChain", "runnables", "are", "powerful!"]
result = join_words_runnable.invoke(input_words)

print(result)
# Expected Output: LangChain runnables are powerful!
```

#### 2. `RunnableSequence`

This is the most common way to build chains. It executes a series of Runnables in order, where the output of one becomes the input for the next. The `|` (pipe) operator is the elegant shorthand for creating a sequence, making the flow of data intuitive and easy to read. Data flows from left to right, often changing its type as it passes through each component.

**Use Case:** Create a simple chain that takes a topic, formats it into a prompt, and gets a response from a model.

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Define the components of our chain
prompt_template = ChatPromptTemplate.from_template("Tell me a fact about {topic}.")
model = ChatOpenAI() # Note: Requires OPENAI_API_KEY to be set
output_parser = StrOutputParser()

# Create the sequence using the pipe operator
# Data flow: dict -> PromptValue -> AIMessage -> str
chain = prompt_template | model | output_parser

# Execute the chain
result = chain.invoke({"topic": "the moon"})

print(result)
# Expected Output: A fact about the moon, e.g., "The Moon is drifting away from the Earth at a rate of about 3.8 cm per year."
```

#### 3. `RunnableParallel`

This allows you to execute multiple Runnables at the same time with the *same input*. The results are returned in a dictionary, where the keys are the ones you define. It's great for when you need to run independent operations on the same piece of data, often leading to significant performance gains as the operations can be run concurrently.

**Use Case:** You have a user's question and you want to simultaneously get a direct answer from an LLM and also generate some potential follow-up questions.

```python
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# A chain to get a direct answer
answer_chain = (
    ChatPromptTemplate.from_template("Answer the following question: {question}")
    | ChatOpenAI()
)

# A chain to generate follow-up questions
follow_up_chain = (
    ChatPromptTemplate.from_template("Generate 3 follow-up questions for: {question}")
    | ChatOpenAI()
)

# Create a parallel runnable. The keys 'answer' and 'follow_up' will be the keys in the output dictionary.
# Both chains will receive the same input: {"question": "..."}
combined_chain = RunnableParallel(
    answer=answer_chain,
    follow_up=follow_up_chain
)

# Execute the parallel chain
result = combined_chain.invoke({"question": "What is the capital of France?"})

print(result)
# Expected Output:
# {
#   'answer': AIMessage(content='The capital of France is Paris.'),
#   'follow_up': AIMessage(content='1. What are some famous landmarks in Paris?\n2. What is the history of the Eiffel Tower?\n3. What is the official language spoken in France?')
# }
```

#### 4. `RunnablePassthrough`

This is a simple but very useful utility. It takes the input and passes it through unchanged. It's often used with `RunnableParallel` to pass along an original input field alongside new, computed values. This is crucial for maintaining context that might be needed by later steps in a more complex chain.

**Use Case:** You want to answer a question but also keep the original question in the final output for context.

```python
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# The main chain that will answer the question
answer_chain = (
    ChatPromptTemplate.from_template("Tell me a fact about {topic}.")
    | ChatOpenAI()
    | StrOutputParser()
)

# We use RunnablePassthrough to pass the original 'topic' through.
# The 'answer' key will be populated by our answer_chain.
chain = RunnableParallel(
    answer=answer_chain,
    original_topic=RunnablePassthrough()
)

# Execute the chain
result = chain.invoke({"topic": "the sun"})

print(result)
# Expected Output:
# {
#   'answer': 'The Sun accounts for about 99.86% of the total mass of the Solar System.',
#   'original_topic': {'topic': 'the sun'}
# }
```

### The Runnable Interface: `invoke`, `batch`, and `stream`

This is the standard API for executing any Runnable. Having a consistent API means you don't have to think about *how* to run a component; you only need to decide whether you're processing a single item, a batch of items, or streaming the output.

#### `invoke()`: Single Input, Single Output

This is the most straightforward method. You use it when you have one input and you want to get a single, complete output back. It's a synchronous, blocking call; your code will wait until the entire operation is finished before moving to the next line.

**Use Case:** A simple translation task.

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Translate '{text}' from English to French.")
model = ChatOpenAI()

# Create a simple chain
translation_chain = prompt | model

# Use invoke for a single input/output operation
result = translation_chain.invoke({"text": "I love programming."})

print(result.content)
# Expected Output: "J'adore la programmation."
```

#### `batch()`: Multiple Inputs, Multiple Outputs

This is for efficiency. When you have a list of inputs, `batch()` processes them in parallel (where possible), which is much faster than calling `invoke()` in a loop. LangChain's runtime can make concurrent API calls to the model provider, dramatically reducing the total execution time for large datasets.

**Use Case:** Translating a list of phrases.

```python
# (Continuing from the previous example)

# A list of inputs
phrases = [
    {"text": "Hello world"},
    {"text": "How are you?"},
    {"text": "Good morning"}
]

# Use batch to process all inputs at once, which is much faster than a for loop.
results = translation_chain.batch(phrases)

for res in results:
    print(res.content)

# Expected Output:
# "Bonjour le monde"
# "Comment allez-vous ?"
# "Bonjour"
```

#### `stream()`: Chunked Outputs

This is for real-time applications, like chatbots. Instead of waiting for the full response, `stream()` returns an iterator that yields the output in chunks as it's being generated by the model. This creates a much better user experience, as the user sees immediate feedback instead of a loading spinner, improving the perceived performance of the application.

**Use Case:** Streaming a story from an LLM so the user sees the text appearing word-by-word.

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a short story about a brave knight named {name}.")
model = ChatOpenAI()
parser = StrOutputParser()

story_chain = prompt | model | parser

# Use stream to get the output in chunks
# The 'for' loop will print each chunk as it arrives, giving a real-time effect.
for chunk in story_chain.stream({"name": "Arthur"}):
    print(chunk, end="", flush=True)

# Expected Output: A story about Sir Arthur will be printed to the console
# token by token, giving a real-time streaming effect.
```
